# 1. Install Dependencies and Setup

In [1]:
print("Hello, world!")

Hello, world!


In [2]:
import tensorflow as tf
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [3]:
imgsize = 1024
batchsize = 16
data_dir = 'data2'
epochs = 31

In [4]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# 3. Load and Scale Data

In [6]:
data = tf.keras.utils.image_dataset_from_directory(data_dir, batch_size=batchsize, image_size=(imgsize, imgsize))

Found 2024 files belonging to 2 classes.


In [7]:
data = data.map(lambda x,y: (x/255, y))

# 4. Split Data

In [8]:
len(data)

127

In [14]:
train_size = int(len(data)*.7)+1
val_size = int(len(data)*.2)+1
test_size = int(len(data)*.1)

In [15]:
print("Train size: ", train_size)
print("Val size: ", val_size)
print("Test size: ", test_size)
print("Total size: ", train_size+val_size+test_size)

Train size:  89
Val size:  26
Test size:  12
Total size:  127


In [16]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

# 5. Build Deep Learning Model

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [18]:
model = Sequential()

In [19]:
model.add(Conv2D(32, (7,7), 1, activation='relu', input_shape=(imgsize,imgsize,3)))
model.add(MaxPooling2D())

model.add(Conv2D(64, (5,5), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(128, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(256, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [20]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1018, 1018, 32)    4736      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 509, 509, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 505, 505, 64)      51264     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 252, 252, 64)     0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 250, 250, 128)     73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 125, 125, 128)    0

# 6. Train

In [22]:
logdir='logs'

In [23]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=epochs, validation_data=val, callbacks=[tensorboard_callback])

Epoch 1/31
89/89 [==============================] - 43s 409ms/step - loss: 0.9047 - accuracy: 0.7451 - val_loss: 0.2357 - val_accuracy: 0.8870
Epoch 2/31
89/89 [==============================] - 36s 403ms/step - loss: 0.4009 - accuracy: 0.8371 - val_loss: 0.2613 - val_accuracy: 0.8774
Epoch 3/31
61/89 [===================>..........] - ETA: 8s - loss: 0.2357 - accuracy: 0.9068

# 7. Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# 8. Evaluate

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

# 9. Test

In [ ]:
img0 = cv2.imread('test/897.jpg')
img1 = cv2.imread('test/1253.jpg')
img2 = cv2.imread('test/1254.jpg')
img3 = cv2.imread('test/1305.jpg')
img4 = cv2.imread('test/1470.jpg')

img5 = cv2.imread('test/1547.jpg')
img6 = cv2.imread('test/1430.jpg')
img7 = cv2.imread('test/1431.jpg')
img8 = cv2.imread('test/1466.jpg')
img9 = cv2.imread('test/1515.jpg')
#plt.imshow(cv2.cvtColor(img0, cv2.COLOR_BGR2RGB))
#plt.show()

In [ ]:
resize0 = tf.image.resize(img0, (imgsize,imgsize))
resize1 = tf.image.resize(img1, (imgsize,imgsize))
resize2 = tf.image.resize(img2, (imgsize,imgsize))
resize3 = tf.image.resize(img3, (imgsize,imgsize))
resize4 = tf.image.resize(img4, (imgsize,imgsize))

resize5 = tf.image.resize(img5, (imgsize,imgsize))
resize6 = tf.image.resize(img6, (imgsize,imgsize))
resize7 = tf.image.resize(img7, (imgsize,imgsize))
resize8 = tf.image.resize(img8, (imgsize,imgsize))
resize9 = tf.image.resize(img9, (imgsize,imgsize))
#plt.imshow(resize0.numpy().astype(int))
#plt.show()

In [ ]:
#yhat0 = model.predict(np.expand_dims(resize0/255, 0))
yhat0 = model.predict(np.expand_dims(resize0/255, 0))[0][0]
yhat1 = model.predict(np.expand_dims(resize1/255, 0))[0][0]
yhat2 = model.predict(np.expand_dims(resize2/255, 0))[0][0]
yhat3 = model.predict(np.expand_dims(resize3/255, 0))[0][0]
yhat4 = model.predict(np.expand_dims(resize4/255, 0))[0][0]

yhat5 = model.predict(np.expand_dims(resize5/255, 0))[0][0]
yhat6 = model.predict(np.expand_dims(resize6/255, 0))[0][0]
yhat7 = model.predict(np.expand_dims(resize7/255, 0))[0][0]
yhat8 = model.predict(np.expand_dims(resize8/255, 0))[0][0]
yhat9 = model.predict(np.expand_dims(resize9/255, 0))[0][0]

In [ ]:
# should be 1
print("{:.11f}".format(yhat0), " - Want 1.0")
print("{:.11f}".format(yhat1), " - Want 1.0")
print("{:.11f}".format(yhat2), " - Want 1.0")
print("{:.11f}".format(yhat3), " - Want 1.0")
print("{:.11f}".format(yhat4), " - Want 1.0")
print("------------------------")
print("{:.11f}".format(yhat5), " - Want 0.0")
print("{:.11f}".format(yhat6), " - Want 0.0")
print("{:.11f}".format(yhat7), " - Want 0.0")
print("{:.11f}".format(yhat8), " - Want 0.0")
print("{:.11f}".format(yhat9), " - Want 0.0")

# 11. Save the Model

In [ ]:
#from tensorflow.keras.models import load_model

In [ ]:
#model.save(os.path.join('models','imageclassifier.h5'))

In [ ]:
#new_model = load_model(os.path.join('models', 'imageclassifier.h5'))

In [ ]:
#yhatnew = new_model.predict(np.expand_dims(resize0/255, 0))

In [ ]:
#yhatnew

# 12. Release Memory

In [ ]:
from keras import backend as kbe
kbe.clear_session()

In [ ]:
from numba import cuda
cuda.select_device(0)
cuda.close()